In [1]:
# import required packages
# from fastai2.tabular.all import *
import xgboost as xgb
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from helpers import *

pd.options.display.max_columns = None

In [2]:
# path to project directory
path = Path('./')

In [3]:
# read in training dataset
train_df = pd.read_csv(path/'data/train_v7.csv', index_col=0, dtype={'season':str,
                                                                     'squad':str,
                                                                     'comp':str})

In [4]:
train_df.shape

(114090, 58)

In [5]:
# features for model
season = '2021'
cat_vars = ['season', 'position', 'was_home']
cont_vars = ['gw', 'minutes']
dep_var = ['total_points']

# find the latest gameweek
last_gw = train_df['gw'][train_df['season'] == season].max()

if np.isnan(last_gw): 
    next_gw = 1 
else: 
    next_gw = last_gw + 1
    
next_gw

38

In [6]:
# read in remaining_season.csv and sort
remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
                                  index_col=0).sort_values(by=['gw', 'team'])

# update fields needed for processing
remaining_season_df['season'] = season
remaining_season_df['total_points'] = 0

# need to add an ordered date-like field  
remaining_season_df['kickoff_time'] = [str(9000 + x) + '-99-99' for x in remaining_season_df['match_no']]

# remove match_no, don't need it anymore
remaining_season_df = remaining_season_df.drop('match_no', axis=1)

# # append remaining season to end of training set
train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [7]:
# # read in remaining_season.csv and sort
# remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
#                                   index_col=0).sort_values(by=['gw', 'team'])

# remaining_season_df['season'] = season
# remaining_season_df['total_points'] = 0
# remaining_season_df['kickoff_time'] = '9999-99-99'

# # append remaining season to end of training set
# train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [8]:
# add player lag features
lag_train_df, team_lag_vars = team_lag_features(train_df, ['total_points'], ['all', 3, 5, 10, 20])
lag_train_df, player_lag_vars = player_lag_features(lag_train_df, ['total_points'], ['all', 1, 2, 3, 5, 10, 20])

# add team conceded fields for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars += [x.replace('team', 'team_conceded') for x in team_lag_vars if pattern.match(x)]

# but remove scored for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars = [x for x in team_lag_vars if not pattern.match(x)]

In [9]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [10]:
# we want to set gw and season as ordered categorical variables
# need lists with ordered categories
ordered_gws = list(range(1,39))
ordered_seasons = ['1617', '1718', '1819', '1920', '2021']

# set as categories with correct order 
lag_train_df['gw'] = lag_train_df['gw'].astype('category')
lag_train_df['season'] = lag_train_df['season'].astype('category')

lag_train_df['gw'].cat.set_categories(ordered_gws, ordered=True, inplace=True)
lag_train_df['season'].cat.set_categories(ordered_seasons, ordered=True, inplace=True)

In [11]:
lag_train_df[lag_train_df['player'] == 'Bernd Leno'].tail(25)

player  gw  position  minutes     team             opponent_team  \
97918   Bernd Leno  14         1     90.0  Arsenal                   Everton   
98530   Bernd Leno  15         1     90.0  Arsenal                   Chelsea   
99123   Bernd Leno  16         1     90.0  Arsenal  Brighton and Hove Albion   
99622   Bernd Leno  17         1     90.0  Arsenal      West Bromwich Albion   
100175  Bernd Leno  18         1     90.0  Arsenal            Crystal Palace   
100634  Bernd Leno  19         1     90.0  Arsenal          Newcastle United   
101553  Bernd Leno  20         1     90.0  Arsenal               Southampton   
102198  Bernd Leno  21         1     90.0  Arsenal         Manchester United   
102849  Bernd Leno  22         1     71.0  Arsenal   Wolverhampton Wanderers   
103512  Bernd Leno  23         1      0.0  Arsenal               Aston Villa   
104205  Bernd Leno  24         1     90.0  Arsenal                     Leeds   
104987  Bernd Leno  25         1     90.0  Arsenal           Manchester City   
105779  Bernd Leno  26         1     90.0  Arsenal            Leicester City   
106886  Bernd Leno  27         1     90.0  Arsenal                   Burnley   
107633  Bernd Leno  28         1     90.0  Arsenal         Tottenham Hotspur   
108272  Bernd Leno  29         1     90.0  Arsenal           West Ham United   
108596  Bernd Leno  30         1     90.0  Arsenal                 Liverpool   
109283  Bernd Leno  31         1     90.0  Arsenal          Sheffield United   
109976  Bernd Leno  32         1      0.0  Arsenal                    Fulham   
110645  Bernd Leno  33         1     90.0  Arsenal                   Everton   
111208  Bernd Leno  34         1      0.0  Arsenal          Newcastle United   
111884  Bernd Leno  35         1     90.0  Arsenal      West Bromwich Albion   
111885  Bernd Leno  35         1     90.0  Arsenal                   Chelsea   
113467  Bernd Leno  37         1     90.0  Arsenal            Crystal Palace   
114097  Bernd Leno  38         1     72.0  Arsenal  Brighton and Hove Albion   

        relative_market_value_team  relative_market_value_opponent_team  \
97918                     1.317380                             1.145049   
98530                     1.317380                             1.859440   
99123                     1.316625                             0.541862   
99622                     1.261632                             0.258537   
100175                    1.264284                             0.475724   
100634                    1.272515                             0.604033   
101553                    1.343890                             0.538945   
102198                    1.353529                             1.679166   
102849                    1.253765                             0.969065   
103512                    1.253765                             0.749770   
104205                    1.252915                             0.439832   
104987                    1.252915                             2.397273   
105779                    1.252915                             1.059550   
106886                    1.252936                             0.342974   
107633                    1.279916                             1.586900   
108272                    1.280364                             0.652307   
108596                    1.280364                             2.355067   
109283                    1.280737                             0.356395   
109976                    1.280438                             0.520244   
110645                    1.280438                             1.083161   
111208                    1.280464                             0.589279   
111884                    1.279598                             0.282556   
111885                    1.279598                             1.817212   
113467                    1.279628                             0.473959   
114097                    1.279628                      

In [12]:
# create dataset with latest lag numbers for all future gameweeks
valid_season = '2021'
valid_gw = next_gw
valid_len = 38 - next_gw + 1

train_predict_df, train_idx, valid_idx = create_lag_train(lag_train_df, 
                                                      cat_vars, cont_vars, 
                                                      player_lag_vars, team_lag_vars, dep_var,
                                                      valid_season, valid_gw, valid_len)

In [13]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [14]:
# take a look at the dataframe
train_predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114796 entries, 0 to 114795
Data columns (total 33 columns):
 #   Column                                           Non-Null Count   Dtype   
---  ------                                           --------------   -----   
 0   gw                                               114796 non-null  category
 1   minutes                                          114796 non-null  float64 
 2   minutes_last_1                                   114796 non-null  float64 
 3   minutes_last_10                                  114796 non-null  float64 
 4   minutes_last_2                                   114796 non-null  float64 
 5   minutes_last_20                                  114796 non-null  float64 
 6   minutes_last_3                                   114796 non-null  float64 
 7   minutes_last_5                                   114796 non-null  float64 
 8   minutes_last_all                                 114796 non-null  float64 
 9   oppo

In [15]:
# split out dependent variable
X, y = train_predict_df[cat_vars + cont_vars + player_lag_vars + team_lag_vars].copy(), train_predict_df[dep_var].copy()

In [16]:
# since position is categorical, it should be a string
X['position'] = X['position'].apply(str)

# need to transform season
enc = LabelEncoder()
X['season'] = enc.fit_transform(X['season'])
X_dict = X.to_dict("records")

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False, separator='_')

# Apply dv on df: df_encoded
X_encoded = dv.fit_transform(X_dict)

X_df = pd.DataFrame(X_encoded, columns=dv.feature_names_)

In [17]:
# split out training and prediction sets
X_train = X_df.iloc[train_idx]
y_train = y.iloc[train_idx]
X_test = X_df.iloc[valid_idx]
y_test = y.iloc[valid_idx]

In [18]:
# instatiate and train XGB Regressor
model_params = {"gamma": 0.42,
                "learning_rate": 0.047, 
                "max_depth": 4,
                "n_estimators": 171,
                "subsample": 0.6}

xg_reg = xgb.XGBRegressor(objective="reg:squarederror",
                          gamma=model_params['gamma'], 
                          learning_rate=model_params['learning_rate'], 
                          max_depth=model_params['max_depth'],
                          n_estimators=model_params['n_estimators'],
                          subsample=model_params['subsample'])

xg_reg.fit(X_train, y_train)
preds = xg_reg.predict(X_test)

In [19]:
preds

array([ 1.52933300e-02, -2.60931253e-03,  1.09928977e+00,  4.22396326e+00,
        9.35444236e-03,  8.18771720e-02,  1.52933300e-02,  3.38860226e+00,
        1.10327983e+00,  1.14305413e+00,  3.88216674e-02,  2.54626012e+00,
        2.06455588e-03,  3.16278934e+00,  3.91394615e+00,  1.80496788e+00,
       -2.60931253e-03, -3.45653296e-03,  1.43377817e+00,  3.69515944e+00,
        2.99805760e+00, -2.60931253e-03,  1.62489605e+00, -2.60931253e-03,
       -1.09202266e-02,  1.68872678e+00,  3.24847603e+00,  1.24821496e+00,
        2.90715909e+00, -2.09492445e-03, -2.24620104e-03,  1.12382913e+00,
        1.37187910e+00, -2.60931253e-03,  3.81691194e+00,  2.69279480e-02,
        9.08723712e-01,  7.66378641e-03,  8.27696919e-03,  7.46139884e-03,
        7.66378641e-03,  7.66378641e-03,  9.02527571e-03,  8.27696919e-03,
        1.39252126e+00,  2.81876755e+00,  8.30599070e-01,  1.17692947e-02,
        2.15332317e+00,  2.62829900e+00,  8.27696919e-03,  1.08341670e+00,
        4.07403851e+00,  

In [20]:
remaining_season_df['predicted_points'] = preds

In [21]:
remaining_season_df.head(50)

gw         team             opponent_team  \
0   38      Arsenal  Brighton and Hove Albion   
1   38      Arsenal  Brighton and Hove Albion   
2   38      Arsenal  Brighton and Hove Albion   
3   38      Arsenal  Brighton and Hove Albion   
4   38      Arsenal  Brighton and Hove Albion   
5   38      Arsenal  Brighton and Hove Albion   
6   38      Arsenal  Brighton and Hove Albion   
7   38      Arsenal  Brighton and Hove Albion   
8   38      Arsenal  Brighton and Hove Albion   
9   38      Arsenal  Brighton and Hove Albion   
10  38      Arsenal  Brighton and Hove Albion   
11  38      Arsenal  Brighton and Hove Albion   
12  38      Arsenal  Brighton and Hove Albion   
13  38      Arsenal  Brighton and Hove Albion   
14  38      Arsenal  Brighton and Hove Albion   
15  38      Arsenal  Brighton and Hove Albion   
16  38      Arsenal  Brighton and Hove Albion   
17  38      Arsenal  Brighton and Hove Albion   
18  38      Arsenal  Brighton and Hove Albion   
19  38      Arsenal  Brighton and Hove Albion   
20  38      Arsenal  Brighton and Hove Albion   
21  38      Arsenal  Brighton and Hove Albion   
22  38      Arsenal  Brighton and Hove Albion   
23  38      Arsenal  Brighton and Hove Albion   
24  38      Arsenal  Brighton and Hove Albion   
25  38      Arsenal  Brighton and Hove Albion   
26  38      Arsenal  Brighton and Hove Albion   
27  38      Arsenal  Brighton and Hove Albion   
28  38      Arsenal  Brighton and Hove Albion   
29  38      Arsenal  Brighton and Hove Albion   
30  38      Arsenal  Brighton and Hove Albion   
31  38      Arsenal  Brighton and Hove Albion   
32  38      Arsenal  Brighton and Hove Albion   
33  38      Arsenal  Brighton and Hove Albion   
34  38  Aston Villa                   Chelsea   
35  38  Aston Villa                   Chelsea   
36  38  Aston Villa                   Chelsea   
37  38  Aston Villa                   Chelsea   
38  38  Aston Villa                   Chelsea   
39  38  Aston Villa                   Chelsea   
40  38  Aston Villa                   Chelsea   
41  38  Aston Villa                   Chelsea   
42  38  Aston Villa                   Chelsea   
43  38  Aston Villa                   Chelsea   
44  38  Aston Villa                   Chelsea   
45  38  Aston Villa                   Chelsea   
46  38  Aston Villa                   Chelsea   
47  38  Aston Villa                   Chelsea   
48  38  Aston Villa                   Chelsea   
49  38  Aston Villa                   Chelsea   

                              player  position  price  play_proba  \
0                         Mesut Özil         3    6.7         0.0   
1          Sokratis Papastathopoulos         2    4.8         0.0   
2         David Luiz Moreira Marinho         2    5.4         0.5   
3          Pierre-Emerick Aubameyang         3   11.3         1.0   
4                      Cédric Soares         2    4.6         1.0   
5                Alexandre Lacazette         4    8.4         1.0   
6                   Shkodran Mustafi         2    5.0         0.0   
7                         Bernd Leno         1    5.0         1.0   
8                       Granit Xhaka         3    5.2         1.0   
9                         Pablo Marí         2    4.4         1.0   
10                   Héctor Bellerín         2    5.0         0.0   
11                    Calum Chambers         2    4.5         1.0   
12                    Sead Kolasinac         2    4.8         0.0   
13                       Rob Holding         2    4.3         1.0   
14                    Kieran Tierney         2    5.2         1.0   
15                      Nicolas Pépé         3    7.7         1.0   
16                    Lucas Torreira         3    4.7         0.0   
17                      Reiss Nelson         3    5.1         1.0   
18                    Edward Nketiah         4    5.3         1.0   
19                  Emile Smith Rowe         3    4.2         1.0   
20                       Bukayo Saka         3    5.1  

In [22]:
remaining_season_df[remaining_season_df['gw'] == next_gw].sort_values(by='predicted_points', ascending=False).head(50)

gw               team             opponent_team  \
190  38          Liverpool            Crystal Palace   
501  38  Tottenham Hotspur            Leicester City   
187  38          Liverpool            Crystal Palace   
194  38          Liverpool            Crystal Palace   
229  38    Manchester City                   Everton   
503  38  Tottenham Hotspur            Leicester City   
292  38    West Ham United               Southampton   
610  38            Burnley          Sheffield United   
679  38  Manchester United   Wolverhampton Wanderers   
150  38     Leicester City         Tottenham Hotspur   
133  38              Leeds      West Bromwich Albion   
114  38              Leeds      West Bromwich Albion   
115  38              Leeds      West Bromwich Albion   
282  38    West Ham United               Southampton   
158  38     Leicester City         Tottenham Hotspur   
188  38          Liverpool            Crystal Palace   
223  38    Manchester City                   Everton   
112  38              Leeds      West Bromwich Albion   
507  38  Tottenham Hotspur            Leicester City   
426  38   Newcastle United                    Fulham   
191  38          Liverpool            Crystal Palace   
185  38          Liverpool            Crystal Palace   
203  38          Liverpool            Crystal Palace   
451  38   Newcastle United                    Fulham   
234  38    Manchester City                   Everton   
443  38   Newcastle United                    Fulham   
3    38            Arsenal  Brighton and Hove Albion   
113  38              Leeds      West Bromwich Albion   
579  38            Everton           Manchester City   
640  38        Southampton           West Ham United   
419  38            Chelsea               Aston Villa   
692  38  Manchester United   Wolverhampton Wanderers   
52   38        Aston Villa                   Chelsea   
410  38            Chelsea               Aston Villa   
80   38             Fulham          Newcastle United   
427  38   Newcastle United                    Fulham   
249  38   Sheffield United                   Burnley   
304  38    West Ham United               Southampton   
408  38            Chelsea               Aston Villa   
303  38    West Ham United               Southampton   
425  38   Newcastle United                    Fulham   
279  38   Sheffield United                   Burnley   
14   38            Arsenal  Brighton and Hove Albion   
307  38    West Ham United               Southampton   
125  38              Leeds      West Bromwich Albion   
496  38  Tottenham Hotspur            Leicester City   
411  38            Chelsea               Aston Villa   
577  38            Everton           Manchester City   
34   38        Aston Villa                   Chelsea   
523  38  Tottenham Hotspur            Leicester City   

                            player  position  price  play_proba  \
190                  Mohamed Salah         3   12.9         1.0   
501                     Harry Kane         4   11.9         1.0   
187                     Sadio Mané         3   11.8         1.0   
194         Trent Alexander-Arnold         2    7.8         1.0   
229      Gabriel Fernando de Jesus         4    9.1         1.0   
503                  Heung-Min Son         3    9.6         1.0   
292                Michail Antonio         4    6.7         1.0   
610                     Chris Wood         4    6.5         1.0   
679  Bruno Miguel Borges Fernandes         3   11.4         1.0   
150                    Jamie Vardy         4   10.2         1.0   
133           Raphael Dias Belloli         3    5.6         1.0   
114                Patrick Bamford         4    6.6         1.0   
115                  Jack Harrison         3    5.6         1.0   
282                  Jesse Lingard         3    6.6         1.0   
158              Kelechi Iheanacho         4    6.2         1.0   
188          Alisson Ramses Becker         1    6.0         1.0   
223      

In [23]:
# rename previous week's predictions file
# generate previous week's filename
last_gw = next_gw - 1
filename = 'history/2020-21/predictions_gw' + str(last_gw) + '.csv'
# windows
! move predictions.csv $filename
# linux
# ! mv predictions.csv $filename

        1 file(s) moved.


In [24]:
# write to predictions.csv
## RENAME PREVIOUS WEEK FIRST
predictions = remaining_season_df[['gw', 'player', 'team', 'opponent_team', 
                                   'position', 'price', 'play_proba', 'short_name',
                                   'was_home', 'predicted_points', 'minutes']]
predictions.to_csv('predictions.csv')